In [ ]:
import random
from pathlib import Path
from math import ceil, log2, sqrt
import pprint

import numpy as np
import matplotlib.pyplot as plt

# https://piazza.com/class/ksmlgkord5752e?cid=2325
MAX_LENGTH = 1024
# Because the output must contain the first and last 50 letters of the actual
# alignment.
MIN_LENGTH = 50

INPUT_DIR = Path('input')
BASE_STR_LENGTHS = 4, 5, 6, 7
BASES = 'ACTG' * 2, 'TACG' * 2
N_DATA_POINTS = 20
N_REPEATS = 3

In [ ]:
# Choose string generation parameters.
def choose_str_generation_parameters(l: float) -> tuple[int, int, int]:
    """
    :param l string length to approximate
    :return (base string length, number of duplicate operations, generated string length)
    """
    fractional_powers = tuple(log2(l / m) for m in BASE_STR_LENGTHS)
    powers = tuple(ceil(x) for x in fractional_powers)
    i = np.argmin(tuple(x - y for x, y in zip(powers, fractional_powers)))
    return BASE_STR_LENGTHS[i], powers[i], BASE_STR_LENGTHS[i] * 2 ** powers[i]


# Special handling for the shortest strings such that they are *both* at least
# MIN_LENGTH long.
str_gen_parameters = [(choose_str_generation_parameters(MIN_LENGTH),) * 2]
target_problem_sizes = np.geomspace(MIN_LENGTH * 2, MAX_LENGTH * 2, num=N_DATA_POINTS)
for size in target_problem_sizes[1:]:
    t = choose_str_generation_parameters(size / 2)
    str_gen_parameters.append((t, choose_str_generation_parameters(size - t[2])))
# Print its repr for caching in the cell below.
pprint.pprint(str_gen_parameters)

In [ ]:
str_gen_parameters = [((7, 3, 56), (7, 3, 56)),
 ((4, 4, 64), (7, 3, 56)),
 ((5, 4, 80), (4, 4, 64)),
 ((6, 4, 96), (5, 4, 80)),
 ((6, 4, 96), (6, 4, 96)),
 ((7, 4, 112), (7, 4, 112)),
 ((5, 5, 160), (7, 4, 112)),
 ((5, 5, 160), (5, 5, 160)),
 ((6, 5, 192), (6, 5, 192)),
 ((7, 5, 224), (7, 5, 224)),
 ((4, 6, 256), (4, 6, 256)),
 ((5, 6, 320), (4, 6, 256)),
 ((6, 6, 384), (5, 6, 320)),
 ((7, 6, 448), (6, 6, 384)),
 ((4, 7, 512), (7, 6, 448)),
 ((5, 7, 640), (7, 6, 448)),
 ((5, 7, 640), (5, 7, 640)),
 ((6, 7, 768), (6, 7, 768)),
 ((7, 7, 896), (7, 7, 896)),
 ((4, 8, 1024), (4, 8, 1024))]

In [ ]:
# Generate test cases.
random.seed()
INPUT_DIR.mkdir(exist_ok=True)
for i, pair in enumerate(str_gen_parameters):
    with (INPUT_DIR / f'{i}.txt').open('w') as f:
        for base, (l, n_operations, _) in zip(BASES, pair):
            print(base[:l], file=f)
            for _ in range(n_operations):
                print(random.randrange(l), file=f)
                l *= 2

In [ ]:
# Run the test cases.
for i in range(N_DATA_POINTS):
    print(i)
    f = INPUT_DIR / f'{i}.txt'
    !python3 efficient.py {f}

In [ ]:
# Run the test cases and gather output.
ALGORITHM_VERSIONS = ('basic', 'efficient')
OUTPUT_DIR = Path('output')
OUTPUT_DIR.mkdir(exist_ok=True)
OUTPUT_FILE_NAME = 'output.txt'
for v in ALGORITHM_VERSIONS:
    output_dir = OUTPUT_DIR / v
    output_dir.mkdir(exist_ok=True)
    for i in range(N_DATA_POINTS):
        print(i)
        f = INPUT_DIR / f'{i}.txt'
        !python3 {v}.py {f}         # Warm up.
        
        for j in range(N_REPEATS):
            output_file = output_dir / f'{i}-{j}.txt'
            !python3 {v}.py {f}
            !mv {OUTPUT_FILE_NAME} {output_file}

In [ ]:
subplots = {
    'cpu': plt.subplots(),
    'mem': plt.subplots()
}

x = tuple(t1[2] + t2[2] for t1, t2 in str_gen_parameters)
for v in ALGORITHM_VERSIONS:
    y_cpu = []
    y_mem = []
    for i in range(N_DATA_POINTS):
        lines = tuple(tuple((OUTPUT_DIR / v / f'{i}-{j}.txt').open('r')) for j in range(N_REPEATS))
        y_cpu.append(sum(float(lines[j][3]) for j in range(N_REPEATS)) / N_REPEATS)
        y_mem.append(sum(float(lines[j][4]) for j in range(N_REPEATS)) / N_REPEATS)
    subplots['cpu'][1].plot(x, y_cpu, '-o', label=v)
    subplots['mem'][1].plot(x, y_mem, '-o', label=v)

for sp in subplots.values():
    sp[1].set_xscale('log')
    sp[1].set_yscale('log')
    sp[1].set_xlabel('Problem Size')
    sp[1].legend()
subplots['cpu'][1].set_ylabel('CPU Time/s')
subplots['mem'][1].set_ylabel('Memory/KB')

In [ ]:
subplots['cpu'][0].savefig('CPUPlot.png')
subplots['mem'][0].savefig('MemoryPlot.png')